In [ ]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.1/542.1 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.0/172.0 kB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.5/239.5 kB 21.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.3/124.3 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.6/301.6 kB 26.2 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uni

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import matplotlib.pyplot as plt
from datasets import load_dataset

# Function to load model and tokenizer
def load_model(model_name):
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(model_name)
    return tokenizer, model

# Function to generate completion
def generate_completion(model, tokenizer, prompt, max_length=1000, temperature=0.7):
    inputs = tokenizer(prompt, return_tensors="pt")
    input_ids = inputs["input_ids"]
    attention_mask = inputs["attention_mask"]

    with torch.no_grad():
        outputs = model.generate(input_ids=input_ids, attention_mask=attention_mask,
                                 max_length=max_length, temperature=temperature)

    completion = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return completion

# Evaluation functions for each task
def evaluate_human_eval(model, tokenizer, dataset, num_examples=50):
    correct = 0
    results = []
    for i, example in enumerate(dataset["test"]):
        if i >= num_examples:
            break

        prompt = example["prompt"]
        expected_output = example["canonical_solution"]

        generated_output = generate_completion(model, tokenizer, prompt)

        print(f"Prompt: {prompt}")
        print(f"Expected Output: {expected_output}")
        print(f"Generated Output: {generated_output}")
        print("----------------------------------------------------")

        is_correct = expected_output in generated_output
        results.append(is_correct)
        if is_correct:
            correct += 1

    accuracy = correct / num_examples
    print(f"Accuracy: {accuracy * 100}%")

    return results

def evaluate_near_protocol(model, tokenizer, questions):
    results = []
    for question, expected_answer in questions:
        generated_answer = generate_completion(model, tokenizer, question)

        print(f"Question: {question}")
        print(f"Expected Answer: {expected_answer}")
        print(f"Generated Answer: {generated_answer}")
        print("----------------------------------------------------")

        is_correct = expected_answer in generated_answer
        results.append(is_correct)

    accuracy = sum(results) / len(questions)
    print(f"Accuracy: {accuracy * 100}%")

    return results

def evaluate_dapp_trees(model, tokenizer, prompts):
    results = []
    for prompt, expected_code in prompts:
        generated_code = generate_completion(model, tokenizer, prompt)

        print(f"Prompt: {prompt}")
        print(f"Expected Code: {expected_code}")
        print(f"Generated Code: {generated_code}")
        print("----------------------------------------------------")

        is_correct = expected_code in generated_code
        results.append(is_correct)

    accuracy = sum(results) / len(prompts)
    print(f"Accuracy: {accuracy * 100}%")

    return results

def evaluate_build_near_dapp(model, tokenizer, tasks):
    results = []
    for task, expected_output in tasks:
        generated_output = generate_completion(model, tokenizer, task)

        print(f"Task: {task}")
        print(f"Expected Output: {expected_output}")
        print(f"Generated Output: {generated_output}")
        print("----------------------------------------------------")

        is_correct = expected_output in generated_output
        results.append(is_correct)

    accuracy = sum(results) / len(tasks)
    print(f"Accuracy: {accuracy * 100}%")

    return results

# Plotting function
def plot_results(task, results):
    labels = ['Correct', 'Incorrect']
    counts = [sum(results), len(results) - sum(results)]

    plt.figure(figsize=(8, 6))
    plt.bar(labels, counts, color=['green', 'red'])
    plt.xlabel('Evaluation Result')
    plt.ylabel('Number of Examples')
    plt.title(f'{task} Evaluation Results')
    plt.show()

# Main evaluation routine
def evaluate_models(model_names, human_eval_dataset, near_questions, dapp_prompts, dapp_tasks):
    for model_name in model_names:
        print(f"Evaluating {model_name}")
        tokenizer, model = load_model(model_name)

        # HumanEval
        human_eval_results = evaluate_human_eval(model, tokenizer, human_eval_dataset)
        plot_results('HumanEval', human_eval_results)

        # NEAR Protocol
        near_results = evaluate_near_protocol(model, tokenizer, near_questions)
        plot_results('NEAR Protocol', near_results)

        # dApp Trees
        dapp_trees_results = evaluate_dapp_trees(model, tokenizer, dapp_prompts)
        plot_results('dApp Trees', dapp_trees_results)

        # Building NEAR dApp
        dapp_build_results = evaluate_build_near_dapp(model, tokenizer, dapp_tasks)
        plot_results('Building NEAR dApp', dapp_build_results)

# Load HumanEval dataset
human_eval_dataset = load_dataset("openai_humaneval")

# Example questions, prompts, and tasks
near_questions = [
    ("What is the Nightshade Sharding?", "Nightshade Sharding is a method used by the NEAR protocol for scaling."),
    # Add more questions here...
]

dapp_prompts = [
    ("Generate a smart contract for a simple token on NEAR protocol with appropriate comments.", "Expected code with comments"),
    # Add more prompts here...
]

dapp_tasks = [
    ("Create a basic calendar dApp on NEAR.", "Expected calendar dApp code"),
    ("Create a simple snake game dApp on NEAR.", "Expected snake game code"),
    # Add more tasks here...
]

# List of models to evaluate
model_names = [
    #"jcarbonnell/NEARCoder-3b",
    "bigcode/starcoder2-3b",
    # Add more model names here..."jcarbonnell/NEAR-preTrainedStarCoder2", "jcarbonnell/NEAR-structTunedStarcoder2", "jcarbonnell/NEARCoder-3b"
]

# Run the evaluation
evaluate_models(model_names, human_eval_dataset, near_questions, dapp_prompts, dapp_tasks)


Evaluating bigcode/starcoder2-3b


Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


Prompt: from typing import List


def has_close_elements(numbers: List[float], threshold: float) -> bool:
    """ Check if in given list of numbers, are any two numbers closer to each other than
    given threshold.
    >>> has_close_elements([1.0, 2.0, 3.0], 0.5)
    False
    >>> has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3)
    True
    """

Expected Output:     for idx, elem in enumerate(numbers):
        for idx2, elem2 in enumerate(numbers):
            if idx != idx2:
                distance = abs(elem - elem2)
                if distance < threshold:
                    return True

    return False

Generated Output: from typing import List


def has_close_elements(numbers: List[float], threshold: float) -> bool:
    """ Check if in given list of numbers, are any two numbers closer to each other than
    given threshold.
    >>> has_close_elements([1.0, 2.0, 3.0], 0.5)
    False
    >>> has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3)
    True
    """

def ha

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


Prompt: from typing import List


def separate_paren_groups(paren_string: str) -> List[str]:
    """ Input to this function is a string containing multiple groups of nested parentheses. Your goal is to
    separate those group into separate strings and return the list of those.
    Separate groups are balanced (each open brace is properly closed) and not nested within each other
    Ignore any spaces in the input string.
    >>> separate_paren_groups('( ) (( )) (( )( ))')
    ['()', '(())', '(()())']
    """

Expected Output:     result = []
    current_string = []
    current_depth = 0

    for c in paren_string:
        if c == '(':
            current_depth += 1
            current_string.append(c)
        elif c == ')':
            current_depth -= 1
            current_string.append(c)

            if current_depth == 0:
                result.append(''.join(current_string))
                current_string.clear()

    return result

Generated Output: from typing import List


def 

...

Prompt: 

def encode_cyclic(s: str):
    """
    returns encoded string by cycling groups of three characters.
    """
    # split string to groups. Each of length 3.
    groups = [s[(3 * i):min((3 * i + 3), len(s))] for i in range((len(s) + 2) // 3)]
    # cycle elements in each group. Unless group has fewer elements than 3.
    groups = [(group[1:] + group[0]) if len(group) == 3 else group for group in groups]
    return "".join(groups)


def decode_cyclic(s: str):
    """
    takes as input string encoded with encode_cyclic function. Returns decoded string.
    """

Expected Output:     return encode_cyclic(encode_cyclic(s))

Generated Output: 

def encode_cyclic(s: str):
    """
    returns encoded string by cycling groups of three characters.
    """
    # split string to groups. Each of length 3.
    groups = [s[(3 * i):min((3 * i + 3), len(s))] for i in range((len(s) + 2) // 3)]
    # cycle elements in each group. Unless group has fewer elements than 3.
    groups = [(group[1:] 

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


Prompt: 

def prime_fib(n: int):
    """
    prime_fib returns n-th number that is a Fibonacci number and it's also prime.
    >>> prime_fib(1)
    2
    >>> prime_fib(2)
    3
    >>> prime_fib(3)
    5
    >>> prime_fib(4)
    13
    >>> prime_fib(5)
    89
    """

Expected Output:     import math

    def is_prime(p):
        if p < 2:
            return False
        for k in range(2, min(int(math.sqrt(p)) + 1, p - 1)):
            if p % k == 0:
                return False
        return True
    f = [0, 1]
    while True:
        f.append(f[-1] + f[-2])
        if is_prime(f[-1]):
            n -= 1
        if n == 0:
            return f[-1]

Generated Output: 

def prime_fib(n: int):
    """
    prime_fib returns n-th number that is a Fibonacci number and it's also prime.
    >>> prime_fib(1)
    2
    >>> prime_fib(2)
    3
    >>> prime_fib(3)
    5
    >>> prime_fib(4)
    13
    >>> prime_fib(5)
    89
    """

def prime_fib_list(n: int):
    """
    prime_fib_list return

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


Prompt: 

def triples_sum_to_zero(l: list):
    """
    triples_sum_to_zero takes a list of integers as an input.
    it returns True if there are three distinct elements in the list that
    sum to zero, and False otherwise.

    >>> triples_sum_to_zero([1, 3, 5, 0])
    False
    >>> triples_sum_to_zero([1, 3, -2, 1])
    True
    >>> triples_sum_to_zero([1, 2, 3, 7])
    False
    >>> triples_sum_to_zero([2, 4, -5, 3, 9, 7])
    True
    >>> triples_sum_to_zero([1])
    False
    """

Expected Output:     for i in range(len(l)):
        for j in range(i + 1, len(l)):
            for k in range(j + 1, len(l)):
                if l[i] + l[j] + l[k] == 0:
                    return True
    return False

Generated Output: 

def triples_sum_to_zero(l: list):
    """
    triples_sum_to_zero takes a list of integers as an input.
    it returns True if there are three distinct elements in the list that
    sum to zero, and False otherwise.

    >>> triples_sum_to_zero([1, 3, 5, 0])
    Fal